# OpenMASTER - correlations

-------------------------

In [ ]:
import pyomo.environ as pyo
import openMASTER
import numpy as np
import itertools

### Define the abstract model

In [ ]:
m = openMASTER.make_model()
m

### Model data upload

* If you haven't created the .csv files, please:
    * Be aware the openMASTER_Data.xlsx file has to be downloaded using git-lfs or the following link:
        https://github.com/IIT-EnergySystemModels/openMASTER/raw/main/data/input/openMASTER_Data.xlsx?download=
    * Run the first line of code in this cell, which will both create the .csv files and load them into the DataPortal (this whole function takes several minutes)

* On the contrary, if you have already created the .csv files from the Excel file and haven't changed them, you can directly go on to the second line of code. This will save some minutes.

In any case, add "#" in front of the line you are not running.

In [ ]:
data = openMASTER.load_dataportal_from_excel('../data/input/openMASTER_Data.xlsx')
#data = openMASTER.load_dataportal_from_csv()

### Create the instance of the abstract model 

In [ ]:
instance = m.create_instance(data)

### Principal components analysis (PCA)

In [ ]:
#Correlation set up
instance.f1= instance.f[:instance.pNumPCA]
instance.fr= instance.f[instance.pNumPCA:]



X = instance.pUnc.values
#print(X)

# Compute the mean for each column
s_bar = np.mean(X, axis=0)

# Subtract the mean from each row to center the data
X_o = X - s_bar
#print(X_o)

from sklearn.decomposition import PCA
#we compute the singular value decomposition

u, s, vh = np.linalg.svd(np.cov(X_o.T), full_matrices=True) 
##np.linalg.svd --> Singular Value Decomposition:
##np.cov --> estimate the covariance matrix
##u is eigenvector matrix and s are the eigenvalues

#now we compute the vectors needed to build the uncertainty region
w_max=np.max(np.dot(X_o,u)/np.linalg.norm(u,axis=0),0)
w_max_dict = {f[i]: w_max[i] for i in range(len(f))}

w_min=np.min(np.dot(X_o,u)/np.linalg.norm(u,axis=0),0)
w_min_dict = {f[i]: w_min[i] for i in range(len(f))}

alpha_up=(w_max/np.linalg.norm(u,axis=0))*u
alpha_up_dict = {(f_i, info_j, g_k): alpha_up[i, j] for i, f_i in enumerate(f) for j, (info_j,g_k) in enumerate(itertools.product(info,g))}

alpha_do=(w_min/np.linalg.norm(u,axis=0))*u
alpha_do_dict = {(f_i, info_j, g_k): alpha_do[i, j] for i, f_i in enumerate(f) for j, (info_j,g_k) in enumerate(itertools.product(info,g))}

rest =((w_max+w_min)/(2*np.linalg.norm(u,axis=0)))*u
rest_dict     = {(f_i, info_j, g_k): rest    [i, j] for i, f_i in enumerate(f) for j, (info_j,g_k) in enumerate(itertools.product(info,g))}


print(s_bar)

### Solve the model instance

To solve the model instance, please select a solver within the Pyomo SolverFactory. Please note that any solver has to be previously installed.

In [ ]:
solver = pyo.SolverFactory('gurobi')

solver_results = solver.solve(instance, keepfiles=False, tee=True)

### **Results**

Extract information on variables through the model output to .csv files

In [ ]:
path        = "../data/input/openMASTER_Data.xlsx"
output_path = "../data/tmp/output"
sheetname   = "Output"

d_vars_from_instance = openMASTER.export_model_to_csv(path, output_path, sheetname, instance)

Loading variable information from .csv to a dictionary containing all outputs

In [ ]:
d_vars = openMASTER.import_results_from_csv(output_path)